# Projet maison n° 3

In [5]:
# imports
import pandas as pd
import numpy as np
import re
import os

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [10]:
# names us
def df_names_us():
    list_df = []
    
    for file in os.listdir():
        year = int(re.findall('[0-9]{4}', file)[0])
        df_tmp = pd.read_csv(path + "\\" + file, sep = ",", header = None)
        df_tmp.columns = ['name', 'gender', 'births']
        df_tmp.insert(0, 'year', year)
        list_df.append(df_tmp)

    return pd.concat(list_df, axis = 0, ignore_index = True)

In [11]:
path = 'C:\\Users\\benja\\OneDrive\\Documents\\Telecom\\Kit-data-science\\Session3\\National data'
os.chdir(path)

df_us = df_names_us()
df_us

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2020858,2020,Zykell,M,5
2020859,2020,Zylus,M,5
2020860,2020,Zymari,M,5
2020861,2020,Zyn,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [265]:
# names fr
def df_names_fr(df_us, df_fr):
    df = df_fr.copy()
    df = df.rename(columns = {'annais' : 'year', 'nombre' : 'births', 'preusuel': 'name', 'sexe' : 'gender'})
    df = df.reindex(columns = ['year', 'name', 'gender', 'births'])
    
    df = df.astype(df_us.dtypes.to_dict(), errors = "ignore")
    gender_us = df_us["gender"].unique()
    mapping = {'1' : gender_us[1], '2' : gender_us[0]}
    df["gender"] = df["gender"].astype(str).map(mapping)
    
    df = df.drop(df.loc[df["name"].astype(str).str.len() < 2].index, axis = 0)
    df["name"] = df["name"].astype(str).str.capitalize()
    df = df.drop(df.loc[df["year"].str.contains("[^0-9]{4}")].index, axis = 0)
    df = df.drop(df.loc[df["name"].astype(str).str.contains("[^A-Za-zÀ-ÿ-]")].index, axis = 0)
    
    df = df.sort_values('year')
    df["gender"] = sorted(df["gender"])
    df["name"] = sorted(df["name"])
    df["births"] = sorted(df["births"], reverse = True)
    df = df.reset_index(drop = True)
    return df

In [266]:
file = r'C:\Users\benja\OneDrive\Documents\Telecom\Kit-data-science\Session3\nat2020.csv'

df_fr_raw = pd.read_csv(file, sep = ";")

df_fr = df_names_fr(df_us, df_fr_raw)

df_fr

,year,name,gender,births
0,1900,Aadam,F,53584
1,1900,Aadam,F,52150
2,1900,Aadam,F,51857
3,1900,Aadam,F,51397
4,1900,Aadam,F,50424
...,...,...,...,...
629972,2020,Özlem,M,3
629973,2020,Özlem,M,3
629974,2020,Ümmü,M,3
629975,2020,Ümmü,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [ ]:
# taux de change
def df_taux_change(devises):
    df = pd.DataFrame([], columns=devises)
    return df

### Tests

In [ ]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [ ]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [ ]:
# run tests

run_tests()